# Catalog members
* Bouma+2019

Problem: file is so large that loading the table to RAM crashes my laptop! 


Solution: Dask! But dask does not understand the cds header format so this file should be read into memory first using `ascii.read` with specified formatting, then turn into `pd.DataFrame` and finally into dask `dask.dataframe`.

In [1]:
fp = '../../apjsab4a7et1_mrt.txt'
nrows = sum(1 for _ in open(fp, 'rb'))
nrows

1815328

In [2]:
!ls -lh ../../apjsab4a7et1_mrt.txt

-rw-r--r-- 1 jp jp 902M Jan 16 12:06 ../../apjsab4a7et1_mrt.txt


## parse original published table into smaller csv

### read into memory using ascii.read

In [15]:
from astropy.io import ascii

tab = ascii.read(fp, format="cds",encoding="ISO-8859–1", guess=False)
tab

ID,Cluster,Ref,CName,RAdeg,DEdeg,pmRA,pmDE,plx,Gmag,GBp,GRp,K13,Unique,How,inK13,Com,logt,e_logt
,,,,deg,deg,mas / yr,mas / yr,mas,mag,mag,mag,,,,,,[yr],[yr]
int64,str113,str247,str187,float64,float64,float64,float64,float64,float64,float64,float64,str17,str17,str31,str5,str144,float64,float64
915553884118144,--,Zari_2018_UMS,915553884118144,45.4082,2.16827,-5.33216,-4.48147,2.16337,9.071,9.1287,8.9934,--,--,--,--,--,--,--
1195551392247936,--,Zari_2018_UMS,1195551392247936,43.8815,2.01822,6.07063,-11.7618,6.10161,6.575,6.5825,6.6281,--,--,--,--,--,--,--
1402912413121792,--,Zari_2018_UMS,1402912413121792,44.3752,2.8959,7.40878,-14.3087,2.09202,10.367,10.4273,10.2129,--,--,--,--,--,--,--
1546703623466880,--,Zari_2018_UMS,1546703623466880,42.9593,2.92046,1.5975,1.72112,2.8533,10.0334,10.2293,9.7443,--,--,--,--,--,--,--
1939920764151808,--,Zari_2018_PMS,1939920764151808,45.5998,3.70765,2.25357,-8.90602,8.91902,14.3586,15.9104,13.1393,--,--,--,--,--,--,--
2015099871749760,--,Zari_2018_UMS,2015099871749760,44.8897,3.51755,1.95284,-5.41662,3.38203,8.6851,8.8298,8.4833,--,--,--,--,--,--,--
2058736739430272,--,Zari_2018_UMS,2058736739430272,43.7842,3.48682,29.6576,-12.8301,11.5396,6.8383,7.0073,6.5984,--,--,--,--,--,--,--
2151508033360512,--,Zari_2018_PMS,2151508033360512,45.2943,4.02595,2.56979,-2.30765,2.24184,17.0287,18.4748,15.836,--,--,--,--,--,--,--


### convert into pandas dataframe

In [16]:
df = tab.to_pandas()

### convert to dask dataframe

In [17]:
import dask.dataframe as dd

columns = 'Cluster Ref CName RAdeg DEdeg pmRA pmDE plx Gmag GBp GRp K13 Unique How inK13 Com logt e_logt'.split()
df = dd.from_pandas(df, chunksize=20000)

In [18]:
del tab

In [7]:
df.dtypes

ID           int64
Cluster     object
Ref         object
CName       object
RAdeg      float64
DEdeg      float64
pmRA       float64
pmDE       float64
plx        float64
Gmag       float64
GBp        float64
GRp        float64
K13         object
Unique      object
How         object
inK13       object
Com         object
logt       float64
e_logt     float64
dtype: object

### saving as csv

In [75]:
#1 file with ~44mb
df.compute().to_csv('master_list.csv', index=False)

## plot using hvplot

In [1]:
import pandas as pd

df = pd.read_csv('master_list.csv')
df.shape

(234914, 19)

In [7]:
df

,ID,Cluster,Ref,CName,RAdeg,DEdeg,pmRA,pmDE,plx,Gmag,GBp,GRp,K13,Unique,How,inK13,Com,logt,e_logt
0,915553884118144,NaN,Zari_2018_UMS,915553884118144,45.4082,2.16827,-5.33216,-4.48147,2.16337,9.0710,9.1287,8.9934,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1195551392247936,NaN,Zari_2018_UMS,1195551392247936,43.8815,2.01822,6.07063,-11.76180,6.10161,6.5750,6.5825,6.6281,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1402912413121792,NaN,Zari_2018_UMS,1402912413121792,44.3752,2.89590,7.40878,-14.30870,2.09202,10.3670,10.4273,10.2129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1546703623466880,NaN,Zari_2018_UMS,1546703623466880,42.9593,2.92046,1.59750,1.72112,2.85330,10.0334,10.2293,9.7443,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1939920764151808,NaN,Zari_2018_PMS,1939920764151808,45.5998,3.70765,2.25357,-8.90602,8.91902,14.3586,15.9104,13.1393,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234909,945432910144840064,NGC_2281,CantatGaudin_2018,945432910144840064,101.7020,40.38800,-3.05348,-7.99758,1.86801,10.3710,10.4678,10.2173,NGC_2281,NGC_2281,string_match,False,NaN,8.785,0.022
234910,945436655356176768,NGC_2281,CantatGaudin_2018,945436655356176768,101.5320,40.46900,-3.00449,-8.10515,1.96373,15.3136,16.0581,14.4780,NGC_2281,NGC_2281,string_match,False,NaN,8.785,0.022
234911,945436689715913600,NGC_2281,CantatGaudin_2018,945436689715913600,101.5450,40.47240,-2.95777,-8.16201,1.88332,12.8916,13.2114,12.4145,NGC_2281,NGC_2281,string_match,False,NaN,8.785,0.022
234912,945438957458630528,NGC_2281,CantatGaudin_2018,945438957458630528,101.6210,40.58080,-2.88497,-8.20141,1.94746,16.7202,17.6470,15.7681,NGC_2281,NGC_2281,string_match,False,NaN,8.785,0.022


In [1]:
!ls ../data/TablesBouma2019/apjsab4a7et1_mrt*

../data/TablesBouma2019/apjsab4a7et1_mrt00.txt
../data/TablesBouma2019/apjsab4a7et1_mrt01
../data/TablesBouma2019/apjsab4a7et1_mrt01.txt
../data/TablesBouma2019/apjsab4a7et1_mrt02
../data/TablesBouma2019/apjsab4a7et1_mrt02.txt
../data/TablesBouma2019/apjsab4a7et1_mrt03
../data/TablesBouma2019/apjsab4a7et1_mrt03.txt
../data/TablesBouma2019/apjsab4a7et1_mrt04
../data/TablesBouma2019/apjsab4a7et1_mrt04.txt
../data/TablesBouma2019/apjsab4a7et1_mrt_header.txt
../data/TablesBouma2019/apjsab4a7et1_mrt.txt
